In [1]:
import os
import cv2
import random
import time
from typing import Tuple, Iterator

from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

from config import Config
import common
import custom

2024-08-10 00:07:56.931129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 00:07:56.931218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 00:07:56.931972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-10 00:07:56.937872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-10 00:07:57.802062: W tensorflow/compiler/tf2

In [2]:
model = custom.models.SiameseModel()
model.load_weights("checkpoints/ckpt_112_0.874")

In [3]:
from tensorflow.keras import layers, models

light_augmentation = models.Sequential(
    [
        layers.Rescaling(255),
        layers.Resizing(Config.IMAGE_SHAPE[0], Config.IMAGE_SHAPE[1]),
        custom.augment.RandomZoominAndOut((0.2, 1)),
        layers.RandomTranslation((-0.1, 0.1), (-0.1, 0.1)),
        layers.Rescaling(1.0 / 255),
    ]
)


In [4]:
image_path = common.extract_images(f"dataset/{Config.IMAGE_PATH}")
id_list = list(map(common.get_filename, image_path))

dataset = tf.data.Dataset.from_tensor_slices(image_path)
dataset = dataset.map(common.load_image_to_tensor, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(common.preprocessing, num_parallel_calls=tf.data.AUTOTUNE)

In [5]:
matrix = []
for data in dataset.batch(32):
    pred = model(data)
    matrix.append(pred)
matrix = tf.concat(matrix, axis=0)

2024-08-10 00:08:02.307439: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


In [6]:
matrix.shape

TensorShape([13126, 256])

In [8]:
data = next(iter(dataset))
augment = common.augmentation(data)
data = np.array(augment.numpy() * 255, dtype=np.uint8)
cv2.imwrite("test2.png", data[:, :, ::-1])

True

In [9]:
hit = 0
batch_size = 128
for i, data in enumerate(dataset.batch(batch_size)):
    augment = common.augmentation(data)
    preds = model.predict(augment, batch_size=batch_size, verbose=False)

    for j, pred in enumerate(preds):
        result = tf.math.reduce_sum(tf.math.square(matrix - pred), axis=-1)
        min_index = np.argmin(result.numpy())
        if min_index == i * batch_size + j:
            hit += 1

In [10]:
hit / len(dataset) * 100

4.3653816852049365